# Data Exploration

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.facecolor'] = 'white'

## Load data

In [3]:
raw_csv = 'https://raw.githubusercontent.com/bigdatacup/Big-Data-Cup-2024/main/BDC_2024_Womens_Data.csv'
df = pd.read_csv(raw_csv)

## Add useful columns

In [7]:
def get_status(row):
    '''Determine whether the team making a play is 
        Even Strength (ES)
        Power Play (PP)
        Penalty Kill (PK)
    '''
    if (
        (((row['Home Team'] == row['Team']) &
         (row['Away Team Skaters'] < 6) &
         (row['Home Team Skaters'] < row['Away Team Skaters'])) | 
        ((row['Away Team'] == row['Team']) &
         (row['Home Team Skaters'] < 6) &
         (row['Home Team Skaters'] > row['Away Team Skaters'])))
    ):
        return 'PK'
    elif (
        (((row['Home Team'] == row['Team']) &
         (row['Home Team Skaters'] < 6) &
         (row['Home Team Skaters'] > row['Away Team Skaters'])) | 
        ((row['Away Team'] == row['Team']) &
         (row['Away Team Skaters'] < 6) &
         (row['Home Team Skaters'] < row['Away Team Skaters'])))
    ):
        return 'PP'
    else:
        return 'ES'

In [8]:
# add column with ES, PK, or PP for team making play
df['Team Status'] = df.apply(lambda row : get_status(row), axis=1)

In [9]:
# add column giving an ID to each penalty (or sequence of penalties)
# avoid off-setting penalties
df['Penalty ID'] = (
    (df['Event'] == 'Penalty Taken') & 
    (df['Team Status'] == 'ES') & 
    (df.shift(1)['Event'] != 'Penalty Taken') &
    (df.shift(-1)['Event'] != 'Penalty Taken')
).cumsum()
df.loc[df['Team Status'] == 'ES', 'Penalty ID'] = np.nan
max_id = df['Penalty ID'].max()
print(f"There were {int(max_id)} distinct penalty kills.")

There were 29 distinct penalty kills.


In [10]:
def get_time(clock):
    '''Turn clock time into seconds elapsed'''
    t = clock.split(':')
    return 1200 - (int(t[0]) * 60 + int(t[1]))

In [11]:
# add column with time elapsed at play
df['Time'] = df['Clock'].apply(get_time)

In [12]:
# add columns with flag whether team gained or maintained possession on play
df['Pos Gained'] = (
    (df['Date'].shift(1) != df['Date']) | 
    (df['Period'].shift(1) != df['Period']) | 
    (df['Team'].shift(1) != df['Team']) |
    (df['Event'] == 'Faceoff Win')
)
df['Pos Retained'] = (
    (df['Date'].shift(-1) == df['Date']) & 
    (df['Period'].shift(-1) == df['Period']) & 
    (df['Team'].shift(-1) == df['Team'])
)

In [13]:
# label uninterrupted play sequences 
# i.e. no play stoppage and no change in possession
df['Sequence ID'] = df['Pos Gained'].cumsum()
avg_len = df.groupby('Sequence ID').size().mean().round()
max_len = df.groupby('Sequence ID').size().max()
print(f"The average possession sequence was {avg_len:.0f} plays long and the longest was {max_len}.")

The average possession sequence was 4 plays long and the longest was 27.


In [14]:
# add columns with possession time up to and after play
start_time = df.groupby('Sequence ID')['Time'].min().rename('Start Time')
df = pd.merge(df, start_time, how='outer', on='Sequence ID')
df['Past Possession Time'] = df['Time'] - df['Start Time']
df.drop('Start Time', axis=1, inplace=True)

pos_time = df.groupby('Sequence ID')['Past Possession Time'].max().rename('Possession Time')
df = pd.merge(df, pos_time, how='outer', on='Sequence ID')

avg_t = df.groupby('Sequence ID')['Possession Time'].max().mean()
max_t = df.groupby('Sequence ID')['Possession Time'].max().max()
print(f"The average possession sequence lasted {avg_t:.2f} seconds and the longest lasted {max_t} seconds.")

The average possession sequence lasted 5.11 seconds and the longest lasted 59 seconds.


In [15]:
df.to_csv('data/BDC_2024_Data_Cleaned.csv')